## EfficientNetV2L (https://arxiv.org/pdf/2104.00298v2.pdf)

In [1]:
!pip install tensorflow_addons --user
!pip install tensorflow_datasets --user
!pip install tensorflow_hub
#!pip install tensorflow==2.9.0 --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 15.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
#import tensorflow_addons as tfa
#import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
#import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

import posixpath

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!nvidia-smi

Sun Jul 10 20:21:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:

colab= '/content/drive/MyDrive'
local= r'G:\My Drive'

code_runtime= colab


In [6]:
tf.__version__

'2.8.2'

In [7]:
file1= 'thesis/dataset/paddy_doctor/train.csv'
inp_file = posixpath.join(code_runtime, file1)

train_table=pd.read_csv(inp_file)
train_table

,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45
...,...,...,...,...
10402,107607.jpg,tungro,Zonal,55
10403,107811.jpg,tungro,Zonal,55
10404,108547.jpg,tungro,Zonal,55
10405,110245.jpg,tungro,Zonal,55


In [8]:
labels=list(np.unique(train_table['label']))
labels

['bacterial_leaf_blight',
 'bacterial_leaf_streak',
 'bacterial_panicle_blight',
 'blast',
 'brown_spot',
 'dead_heart',
 'downy_mildew',
 'hispa',
 'normal',
 'tungro']

In [9]:
file2= 'thesis/dataset/paddy_doctor/train_images/'
img_files= posixpath.join(code_runtime, file2)

train_data= tf.keras.utils.image_dataset_from_directory(img_files,
                                                        labels='inferred',
                                                        label_mode='categorical',
                                                        class_names=labels,
                                                        color_mode='rgb',
                                                        image_size=(224,224),
                                                        shuffle=True,
                                                        validation_split= 0.2,
                                                        subset='training',
                                                        batch_size= 128,
                                                        seed= 101)

val_data= tf.keras.utils.image_dataset_from_directory(img_files,
                                                        labels= 'inferred',
                                                        label_mode='categorical',
                                                        class_names=labels,
                                                        color_mode='rgb',
                                                        image_size=(224,224),
                                                        shuffle=True,
                                                        validation_split= 0.2,
                                                        subset='validation',
                                                        batch_size= 128,
                                                        seed= 101)

Found 10407 files belonging to 10 classes.
Using 8326 files for training.
Found 10407 files belonging to 10 classes.
Using 2081 files for validation.


In [17]:
# Creating data augmentation layer

augmentation_layer= keras.Sequential([
    layers.RandomFlip("horizontal", seed= 101),
    layers.RandomZoom(0.2, seed= 101),
    layers.RandomRotation(0.2, seed= 101)
])

In [ ]:
# Checking augmentation

plt.figure(figsize=(10, 10))
for images, _ in train_data.take(1):
  for i in range(9):
    augmented_images = augmentation_layer(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

KeyboardInterrupt: ignored

<Figure size 720x720 with 0 Axes>

In [18]:
augmented_train_data= train_data.map(
  lambda x, y: (augmentation_layer(x, training=True), y))

In [19]:
AUTOTUNE = tf.data.AUTOTUNE
augmented_train_data= augmented_train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data= val_data.cache().prefetch(buffer_size=AUTOTUNE)

## Modeling

The preprocessing logic has been included in the EfficientNetV2 model implementation. Users are no longer required to call this method to normalize the input data. This method does nothing and only kept as a placeholder to align the API surface between old and new version of model.

In [24]:
conv_base = tf.keras.applications.efficientnet_v2.EfficientNetV2L(include_top = False,  weights = 'imagenet', 
                                                                      include_preprocessing=True)
conv_base.trainable = False

for layer in conv_base.layers[-100:]:
    layer.trainable = True

for layer in conv_base.layers:
    print(layer.name, layer.trainable)

input_4 False
rescaling_3 False
stem_conv False
stem_bn False
stem_activation False
block1a_project_conv False
block1a_project_bn False
block1a_project_activation False
block1a_add False
block1b_project_conv False
block1b_project_bn False
block1b_project_activation False
block1b_add False
block1c_project_conv False
block1c_project_bn False
block1c_project_activation False
block1c_add False
block1d_project_conv False
block1d_project_bn False
block1d_project_activation False
block1d_add False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_project_conv False
block2a_project_bn False
block2b_expand_conv False
block2b_expand_bn False
block2b_expand_activation False
block2b_project_conv False
block2b_project_bn False
block2b_add False
block2c_expand_conv False
block2c_expand_bn False
block2c_expand_activation False
block2c_project_conv False
block2c_project_bn False
block2c_add False
block2d_expand_conv False
block2d_expand_bn False
block2d_expand_a

In [25]:
def create_model(n_classes):
    inputs= layers.Input(shape = (224, 224, 3))
    conv_base = tf.keras.applications.efficientnet_v2.EfficientNetV2L(include_top = False, 
                                                                      input_tensor = inputs, weights = 'imagenet', 
                                                                      include_preprocessing=True)

    # Freeze the pretrained weights
    conv_base.trainable = False

    for layer in conv_base.layers[-100:]:
      layer.trainable = True

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name = 'GAP')(conv_base.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation = 'relu', kernel_regularizer='l2', kernel_initializer="he_normal")(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Dense(512, activation = 'relu', kernel_regularizer='l2', kernel_initializer="he_normal")(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Dense(128, activation = 'relu', kernel_regularizer='l2', kernel_initializer="he_normal")(x)
    x = layers.Dropout(0.25)(x)
    
    outputs = layers.Dense(n_classes, activation = 'softmax', name = 'pred')(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name = 'EfficientNetv2-L')
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3), 
        loss = 'categorical_crossentropy', 
        metrics = ['accuracy'])
    
    return model

model= create_model(10)

In [26]:
n_epochs = 100
#batch_size= 128
#steps_per_epoch= 8326 // batch_size
#validation_steps= 2081 // batch_size

In [27]:
%%time

filepath= "weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint= ModelCheckpoint(filepath, monitor= 'val_accuracy', verbose= 1, save_best_only =True, mode= 'max')
callbacks_list= [checkpoint]

history= model.fit(augmented_train_data,
                    epochs= n_epochs,
                    validation_data= val_data,
                    callbacks=callbacks_list,
                    verbose= 1)

Epoch 1/100
66/66 [==============================] - ETA: 0s - loss: 23.0772 - accuracy: 0.6129
Epoch 1: val_accuracy improved from -inf to 0.73570, saving model to weights-improvement-01-0.74.hdf5
66/66 [==============================] - 135s 2s/step - loss: 23.0772 - accuracy: 0.6129 - val_loss: 12.0416 - val_accuracy: 0.7357
Epoch 2/100
66/66 [==============================] - ETA: 0s - loss: 6.7796 - accuracy: 0.8333
Epoch 2: val_accuracy improved from 0.73570 to 0.82605, saving model to weights-improvement-02-0.83.hdf5
66/66 [==============================] - 47s 713ms/step - loss: 6.7796 - accuracy: 0.8333 - val_loss: 3.7391 - val_accuracy: 0.8260
Epoch 3/100
66/66 [==============================] - ETA: 0s - loss: 2.2635 - accuracy: 0.9010
Epoch 3: val_accuracy did not improve from 0.82605
66/66 [==============================] - 44s 672ms/step - loss: 2.2635 - accuracy: 0.9010 - val_loss: 1.7685 - val_accuracy: 0.8208
Epoch 4/100
66/66 [==============================] - ETA: 0s

KeyboardInterrupt: ignored

In [ ]:
model_file= 'thesis/dataset/paddy_doctor/paddy_doctor0_new'
model_loc= posixpath.join(code_runtime, model_file)
model.save(model_loc)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize = (18, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
model = keras.models.load_model("/content/drive/MyDrive/thesis/dataset/paddy_doctor/paddy_doctor0")

In [ ]:
model.summary()